# Notebook 4: Sentiment Analysis

This notebook implements:
- Using pretrained XLM-RoBERTa for sentiment analysis
- Classifying Nepali news as Positive, Negative, or Neutral
- Calculating sentiment scores and confidence
- Visualizing sentiment distribution across categories

In [ ]:
# Import libraries
import json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    pipeline
)
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print("✓ Libraries imported")

## 1. Configuration

In [ ]:
# Paths
BASE_DIR = Path(r'c:\Users\sagun\Desktop\news_project')
DATA_DIR = BASE_DIR / 'data' / 'processed'
MODEL_DIR = BASE_DIR / 'models' / 'sentiment_analyzer'
RESULTS_DIR = BASE_DIR / 'results'

MODEL_DIR.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Pretrained sentiment model (multilingual)
MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

print(f"Pretrained Model: {MODEL_NAME}")

## 2. Load Pretrained Model

In [ ]:
# Load pretrained sentiment analysis model
print(f"Loading pretrained model: {MODEL_NAME}...\n")

sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    device=0 if torch.cuda.is_available() else -1
)

print("✓ Sentiment analysis pipeline loaded")

# Test the model
test_texts = [
    "यो एक राम्रो समाचार हो।",  # This is good news
    "यो एक नराम्रो घटना हो।",  # This is a bad incident
]

print("\nTesting sentiment analysis:")
for text in test_texts:
    result = sentiment_pipeline(text)[0]
    print(f"Text: {text}")
    print(f"Sentiment: {result['label']} (confidence: {result['score']:.4f})\n")

## 3. Load Data

In [ ]:
# Load test data
with open(DATA_DIR / 'test_data.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

df = pd.DataFrame(test_data)
print(f"✓ Loaded {len(df)} articles")

# Use a sample for demonstration
sample_size = min(100, len(df))
sample_df = df.sample(n=sample_size, random_state=42).copy()
print(f"✓ Using {sample_size} articles for sentiment analysis")

## 4. Perform Sentiment Analysis

In [ ]:
def analyze_sentiment(text, max_length=512):
    """
    Analyze sentiment of text using pretrained model
    """
    try:
        # Truncate text if too long
        words = text.split()
        if len(words) > max_length:
            text = ' '.join(words[:max_length])
        
        result = sentiment_pipeline(text)[0]
        return {
            'sentiment': result['label'],
            'confidence': result['score']
        }
    except Exception as e:
        return {
            'sentiment': 'Neutral',
            'confidence': 0.0
        }

# Analyze all articles
print("Analyzing sentiment for all articles...\n")

sentiments = []
for text in tqdm(sample_df['text'], desc="Sentiment Analysis"):
    result = analyze_sentiment(text)
    sentiments.append(result)

# Add results to dataframe
sample_df['sentiment'] = [s['sentiment'] for s in sentiments]
sample_df['sentiment_confidence'] = [s['confidence'] for s in sentiments]

print("\n✓ Sentiment analysis complete!")

## 5. Analyze Results

In [ ]:
# Overall sentiment distribution
print("Sentiment Distribution:\n")
print(sample_df['sentiment'].value_counts())
print(f"\nAverage confidence: {sample_df['sentiment_confidence'].mean():.4f}")

# Sentiment by category
print("\nSentiment by Category:\n")
sentiment_by_category = pd.crosstab(sample_df['category'], sample_df['sentiment'], normalize='index') * 100
print(sentiment_by_category.round(2))

## 6. Visualizations

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Overall sentiment distribution
sentiment_counts = sample_df['sentiment'].value_counts()
colors = {'Positive': '#2ecc71', 'Negative': '#e74c3c', 'Neutral': '#95a5a6'}
sentiment_colors = [colors.get(s, '#95a5a6') for s in sentiment_counts.index]

axes[0, 0].pie(sentiment_counts.values, labels=sentiment_counts.index, autopct='%1.1f%%',
               colors=sentiment_colors, startangle=90)
axes[0, 0].set_title('Overall Sentiment Distribution', fontweight='bold', fontsize=14)

# 2. Sentiment by category
sentiment_by_cat = pd.crosstab(sample_df['category'], sample_df['sentiment'])
sentiment_by_cat.plot(kind='bar', stacked=True, ax=axes[0, 1], 
                      color=['#e74c3c', '#95a5a6', '#2ecc71'])
axes[0, 1].set_title('Sentiment Distribution by Category', fontweight='bold', fontsize=14)
axes[0, 1].set_xlabel('Category')
axes[0, 1].set_ylabel('Count')
axes[0, 1].legend(title='Sentiment')
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Confidence distribution
axes[1, 0].hist(sample_df['sentiment_confidence'], bins=30, color='skyblue', edgecolor='black')
axes[1, 0].set_title('Sentiment Confidence Distribution', fontweight='bold', fontsize=14)
axes[1, 0].set_xlabel('Confidence Score')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].axvline(sample_df['sentiment_confidence'].mean(), color='red', 
                   linestyle='--', label='Mean')
axes[1, 0].legend()

# 4. Average confidence by sentiment
avg_conf = sample_df.groupby('sentiment')['sentiment_confidence'].mean().sort_values()
axes[1, 1].barh(avg_conf.index, avg_conf.values, 
                color=[colors.get(s, '#95a5a6') for s in avg_conf.index])
axes[1, 1].set_title('Average Confidence by Sentiment', fontweight='bold', fontsize=14)
axes[1, 1].set_xlabel('Average Confidence')
axes[1, 1].set_xlim(0, 1)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'sentiment_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✓ Visualization saved to {RESULTS_DIR / 'sentiment_analysis.png'}")

## 7. Save Results

In [ ]:
# Save sentiment scores
sentiment_results = sample_df[['text', 'category', 'sentiment', 'sentiment_confidence']].to_dict('records')

with open(RESULTS_DIR / 'sentiment_scores.json', 'w', encoding='utf-8') as f:
    json.dump(sentiment_results, f, ensure_ascii=False, indent=2)

print(f"✓ Sentiment scores saved to {RESULTS_DIR / 'sentiment_scores.json'}")

# Save statistics
stats = {
    'total_articles': len(sample_df),
    'sentiment_distribution': sample_df['sentiment'].value_counts().to_dict(),
    'average_confidence': float(sample_df['sentiment_confidence'].mean()),
    'sentiment_by_category': sentiment_by_category.to_dict()
}

with open(RESULTS_DIR / 'sentiment_statistics.json', 'w', encoding='utf-8') as f:
    json.dump(stats, f, ensure_ascii=False, indent=2)

print(f"✓ Statistics saved to {RESULTS_DIR / 'sentiment_statistics.json'}")

## 8. Example Results

In [ ]:
# Show examples for each sentiment
print("="*80)
print("EXAMPLE SENTIMENT ANALYSIS RESULTS")
print("="*80)

for sentiment in ['Positive', 'Negative', 'Neutral']:
    examples = sample_df[sample_df['sentiment'] == sentiment].head(2)
    
    print(f"\n{sentiment.upper()} Examples:")
    print("-" * 80)
    
    for idx, row in examples.iterrows():
        print(f"\nCategory: {row['category']}")
        print(f"Text: {row['text'][:200]}...")
        print(f"Sentiment: {row['sentiment']} (confidence: {row['sentiment_confidence']:.4f})")
        print("-" * 80)

## 9. Summary

In [ ]:
print("="*80)
print("SENTIMENT ANALYSIS SUMMARY")
print("="*80)
print(f"\n🤖 Pretrained Model: {MODEL_NAME}")
print(f"📊 Articles Analyzed: {len(sample_df)}")
print(f"\n📈 Sentiment Distribution:")
for sentiment, count in sample_df['sentiment'].value_counts().items():
    percentage = (count / len(sample_df)) * 100
    print(f"  • {sentiment}: {count} ({percentage:.1f}%)")
print(f"\n🎯 Average Confidence: {sample_df['sentiment_confidence'].mean():.4f}")
print(f"\n💾 Saved Files:")
print(f"  • Sentiment scores: {RESULTS_DIR / 'sentiment_scores.json'}")
print(f"  • Statistics: {RESULTS_DIR / 'sentiment_statistics.json'}")
print(f"  • Visualization: {RESULTS_DIR / 'sentiment_analysis.png'}")
print("\n✅ Sentiment analysis completed successfully!")
print("="*80)